In [28]:
from collections import defaultdict

def MAKEFEATS(stack,queue):
    feats=defaultdict(lambda:0)
    if len(stack) > 0 and len(queue) > 0:
        feats[f"w-1{stack[-1][1]} w-0{queue[0][1]}"]=1
        feats[f"w-1{stack[-1][1]} p-0{queue[0][2]}"]=1
        feats[f"p-1{stack[-1][2]} w-0{queue[0][1]}"]=1
        feats[f"p-1{stack[-1][2]} p-0{queue[0][2]}"]=1
    elif len(stack)>1:
        feats[f"w-2{stack[-2][1]} w-1{stack[-1][1]}"]=1
        feats[f"w-2{stack[-2][1]} p-1{stack[-1][2]}"]=1
        feats[f"p-2{stack[-2][2]} w-1{stack[-1][1]}"]=1
        feats[f"p-2{stack[-2][2]} p-1{stack[-1][2]}"]=1
    return feats
    


def SHIFTREDUCE(queue,w_s,w_r,w_l):
    heads=[-1]*(len(queue)+1)
    stack=[(0,"ROOT","ROOT")]
    while len(queue)>0 or len(stack)>1:
        feats=MAKEFEATS(stack,queue)
        s_s=0
        s_l=0
        s_r=0
        for key,value in feats.items():
            s_s=w_s[key]*int(value)
            s_l=w_l[key]*int(value)
            s_r=w_r[key]*int(value)
        if s_s>=s_l and s_s>=s_r and len(queue)>0:
            stack.append(queue.pop(0))
        elif s_l>=s_r:
            heads[stack[-2][0]]=stack[-1][0]
            stack.pop(-2)
        else:
            heads[stack[-1][0]]=stack[-2][0]
            stack.pop(-1)
    return heads

def SHIFTREDUCE_TRAIN(queue,heads,w_s,w_l,w_r):
    stack=[(0,"ROOT","ROOT")]
    unproc=[heads.count(i) for i in range(len(heads))]
    while len(queue)>0 or len(stack)>1:        
        feats=MAKEFEATS(stack,queue)
        s_s=0
        s_l=0
        s_r=0
        for key,value in feats.items():
            s_s+=w_s[key]*int(value)
            s_l+=w_l[key]*int(value)
            s_r+=w_r[key]*int(value)
            
        #ans
        if s_s>=s_l and s_s>=s_r and len(queue)>0:
            ans="SHIFT"
        elif s_l>=s_r:
            ans="left"
        else:
            ans="right"  
        #corr
        if len(stack)<2:
            corr="SHIFT"
            stack.append(queue.pop(0))
        elif heads[stack[-1][0]]==stack[-2][0] and unproc[stack[-1][0]]==0:
            corr="right"
            unproc[stack[-2][0]]-=1
            stack.pop(-1)
        elif heads[stack[-2][0]]==stack[-1][0] and unproc[stack[-2][0]]==0:
            corr="left"
            unproc[stack[-1][0]]-=1
            stack.pop(-2)
        else:
            corr="SHIFT"
            stack.append(queue.pop(0))
                
        if ans!=corr:
            for key,value in feats.items():
                if ans=="SHIFT":
                    w_s[key]-=value
                elif ans=="right":
                    w_r[key]-=value
                else:
                    w_l[key]-=value
                    
                if corr=="SHIFT":
                    w_s[key]+=value
                elif corr=="right":
                    w_r[key]+=value
                else:
                    w_l[key]+=value
            
    
            

with open("mstparser-en-train.dep.txt","r",encoding="utf-8") as mst:
    data=[]
    heads=[-1]
    queue=[]
    w_s=defaultdict(lambda:0)
    w_l=defaultdict(lambda:0)
    w_r=defaultdict(lambda:0)
    for line in mst:
        line=line.strip()
        items=line.split("\t")
        if len(items)==1:
            data.append((queue,heads))
            queue=[]
            heads=[-1]
        else:
            queue.append((int(items[0]),items[1],items[3]))
            heads.append(int(items[6]))      
    iterations=1
    for i in range(iterations):
        for queue,heads in data:
            SHIFTREDUCE_TRAIN(queue,heads,w_s,w_l,w_r)
    
with open("mstparser-en-test.dep.txt","r",encoding="utf-8") as met:
    queue_test=[]
    data_test=[]
    for line in met:
        line=line.strip()
        items=line.split("\t")
        if len(items)==1:
            data_test.append(queue_test)
            queue_test=[]
        else:
            queue_test.append((int(items[0]),items[1],items[3]))
    with open("answer.txt","w",encoding="utf-8") as ans:
        for queue_test in data_test:
            heads_test=SHIFTREDUCE(queue_test,w_s,w_r,w_l)
            heads_test.pop(0)
            for i in range(len(heads_test)):
                h=f"{i+1}\t_\t_\t_\t_\t_\t{heads_test[i]}\t_\n"
                ans.write(h)
            ans.write("\n")